In [1]:
import opstrat as op
import pandas as pd
import numpy as np
import random
from scipy.stats import norm
import matplotlib.pyplot as plt
import math

import json
with open('config.json') as config_file:
     data = json.load(config_file)
print(data)

last_price          = data['Current_Price']
annual_drift        = data['Annual_Drift']
annual_volatility   = data['Annual_Volatiltiy']
time_period         = data['Time_Period']
simu                = data['No_of_days']
r                   = data['rate_return']
sigma               = data['sigma']
price               = data['Current_Price']

df = pd.DataFrame()

last_price_list = [last_price]
diff_list       = []
#tenor_list      = [0.083333333333333333]
count           = 0
count_list      = []
while count < simu:
    count = count + 1
    difference = last_price*annual_drift*time_period + last_price*annual_volatility*(time_period**(0.5))*norm.ppf(random.random())

    last_price = last_price + difference
    tenor      = ((23-count)/(23*12))
    last_price_list.append(last_price)
    diff_list.append(difference)
    count_list.append(count)
#   tenor_list.append(tenor)
last_price_list.pop(-1)
#tenor_list.pop(-1)
df['Count'] = count_list
df['Price'] = last_price_list
df['Change_price'] = diff_list
#df['tenor'] = tenor_list



{'Current_Price': 17837, 'Annual_Drift': 0.07, 'Annual_Volatiltiy': 0.15, 'Time_Period': 0.003968254, 'rate_return': 0.07, 'sigma': 0.1332, 'No_of_days': 23}


In [2]:
df

,Count,Price,Change_price
0,1,17837.000000,115.308726
1,2,17952.308726,195.262384
2,3,18147.571110,5.085047
3,4,18152.656156,-102.180645
4,5,18050.475512,-276.992643
5,6,17773.482868,-110.404275
6,7,17663.078593,196.932612
7,8,17860.011205,-251.547942
8,9,17608.463263,288.516089
9,10,17896.979352,33.000656


In [3]:
#calculating greeks for call option

datalist2 = dict()

K   =   [int(round(price, -2))+800,int(round(price, -2))+500]

for k in K:
    call_option = []
    call_theta  = []
    call_gamma  = []
    call_delta  = []
    call_rho    = []
    call_vega   = [] 
    for last_price,T in zip(df['Price'],range(30,0,-1)):
        bsm=op.black_scholes(K=k, St=last_price, r=7, t=T,v=13.32, type='c')
        call_option.append(bsm['value']['option value'])  
        call_theta.append(bsm['greeks']['theta'])
        call_gamma.append(bsm['greeks']['gamma'])
        call_delta.append(bsm['greeks']['delta'])
        call_rho.append(bsm['greeks']['rho'])
        call_vega.append(bsm['greeks']['vega'])

        datalist2['CE_'+str(k)]      = call_option
        datalist2['theta_'+str(k)]   = call_theta
        datalist2['gamma_'+str(k)]   = call_gamma
        datalist2['delta_'+str(k)]   = call_delta
        datalist2['rho_'+str(k)]     = call_rho
        datalist2['vega_'+str(k)]    = call_vega

df2 =   pd.DataFrame(datalist2)

#new = pd.DataFrame.from_dict(df2)

#Calculating the greeks for put options

datalist3 = dict()

K   =   [int(round(price, -2))-800,int(round(price, -2))-500]

for k in K:
    put_option  = []
    put_theta   = []
    put_gamma   = []
    put_delta   = []
    put_rho     = []
    put_vega    = [] 
    for last_price,T in zip(df['Price'],range(30,0,-1)):
        bsm     = op.black_scholes(K=k, St=last_price, r=7, t=T,v=13.32, type='p')
        put_option.append(bsm['value']['option value'])   
        put_theta.append(bsm['greeks']['theta'])
        put_gamma.append(bsm['greeks']['gamma'])
        put_delta.append(bsm['greeks']['delta'])
        put_rho.append(bsm['greeks']['rho'])
        put_vega.append(bsm['greeks']['vega'])

        datalist3['PE_'+str(k)]     = put_option
        datalist3['theta_'+str(k)]  = put_theta
        datalist3['gamma_'+str(k)]  = put_gamma
        datalist3['delta_'+str(k)]  = put_delta
        datalist3['rho_'+str(k)]    = put_rho
        datalist3['vega_'+str(k)]   = put_vega
df3 = pd.DataFrame(datalist3)

greeks_df = pd.concat([df2,df3],axis = 1)

In [4]:

#Calculating the total premium paid or recieved 

filter_col = [col for col in list(greeks_df) if col.startswith(('PE_','CE_'))]
greeks_df['Total_Prem'] = 0 # initialize value, otherwise throws KeyErrorx
for d in greeks_df[filter_col]:
    greeks_df['Total_Prem'] = greeks_df[filter_col].sum(axis=1)   # Total_PP = total put premium 


#Calculating the total Profit and Loss

greeks_df['Total_P&L'] = greeks_df['Total_Prem'][0] - greeks_df['Total_Prem']

#print('Profit at expiry period',df[-1:]['Total_P&L'])


In [5]:
#Changing the signs of greeks according to options selling and buying

put_sell = ['theta_','gamma_','delta_','rho_','vega_']
for data in put_sell:
    greeks_df[data+str((round(price, -2))-500)] = - greeks_df[data+str((round(price, -2))-500)]

call_sell = ['theta_','gamma_','delta_','rho_','vega_']
for data in call_sell:
    greeks_df[data+str((round(price, -2))+500)] = - greeks_df[data+str((round(price, -2))+500)]

In [6]:
#Calculating greeks for overall data

list1 = ['theta','gamma','delta','rho','vega']

datalist4 = dict()

for data in list1:
    filter_col = [col for col in list(greeks_df) if col.startswith(data)]
    greeks_df['Port_'+ data] = 0 # initialize value, otherwise throws KeyErrorx
    for d in greeks_df[filter_col]:
        greeks_df['Port_'+ data] = greeks_df[filter_col].mean(axis=1)    

final_df = pd.concat([df,greeks_df],axis = 1)

In [7]:
#final_df.to_csv('A.csv')

In [8]:
final_df

,Count,Price,Change_price,CE_18600,theta_18600,gamma_18600,delta_18600,rho_18600,vega_18600,CE_18300,...,delta_17300,rho_17300,vega_17300,Total_Prem,Total_P&L,Port_theta,Port_gamma,Port_delta,Port_rho,Port_vega
0,1,17837.000000,115.308726,63.943255,-3.539789,0.000381,0.176933,2.541375,13.273878,131.781291,...,0.165962,2.483316,-12.741743,280.645999,0.000000,0.591451,-0.000073,-0.010445,-0.131531,-2.525946
1,2,17952.308726,195.262384,82.834423,-4.164470,0.000438,0.218551,3.051485,14.926311,165.247489,...,0.124516,1.809655,-10.388532,305.604213,-24.958214,0.578582,-0.000066,-0.018779,-0.246105,-2.260683
2,3,18147.571110,5.085047,129.180548,-5.246566,0.000525,0.307315,4.179159,17.665126,241.148737,...,0.072079,1.020146,-6.902167,398.951232,-118.305233,0.466689,-0.000046,-0.030529,-0.400676,-1.540926
3,4,18152.656156,-102.180645,125.463256,-5.300572,0.000532,0.304694,3.998622,17.286051,237.152564,...,0.068237,0.931115,-6.499982,388.752182,-108.106183,0.477496,-0.000047,-0.031655,-0.401823,-1.513459
4,5,18050.475512,-276.992643,92.310470,-4.727746,0.000492,0.246501,3.103718,15.194498,185.596325,...,0.088099,1.151636,-7.700001,312.673885,-32.027886,0.582546,-0.000062,-0.026928,-0.326382,-1.931160
5,6,17773.482868,-110.404275,37.988080,-2.974816,0.000333,0.125292,1.499236,9.590954,89.618206,...,0.176309,2.187337,-12.047241,208.992618,71.653381,0.713455,-0.000091,-0.005508,-0.051642,-2.623856
6,7,17663.078593,196.932612,23.742254,-2.274057,0.000262,0.086696,0.991286,7.152907,61.255938,...,0.223647,2.649467,-13.537348,193.652781,86.993218,0.710205,-0.000097,0.005170,0.074354,-2.637448
7,8,17860.011205,-251.547942,43.208355,-3.418371,0.000379,0.143320,1.585730,10.138500,102.403129,...,0.135357,1.549477,-9.751759,200.342815,80.303184,0.763452,-0.000093,-0.013668,-0.140049,-2.489347
8,9,17608.463263,288.516089,15.563653,-1.829947,0.000215,0.063029,0.659565,5.351634,44.419291,...,0.246417,2.666937,-13.632170,177.035388,103.610611,0.728929,-0.000103,0.011864,0.138446,-2.553387
9,10,17896.979352,33.000656,41.529137,-3.569206,0.000396,0.143683,1.455599,9.724346,102.003449,...,0.114321,1.195709,-8.297411,185.120770,95.525229,0.815222,-0.000097,-0.017848,-0.171780,-2.389970
